In [ ]:
import zipfile
import os
import geopandas as gpd

# File path to the ZIP file containing the train station shapefile
zip_file_path = '/home/Daniel Bi/project two/data/landing/PTV.zip'
extract_dir = '/home/Daniel Bi/project two/data/landing/PTV_train_station'

# Step 1: Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print(f"Files extracted to: {extract_dir}")

# Step 2: List the files in the 'PTV' subdirectory to check for the correct shapefile name
ptv_subdir = os.path.join(extract_dir, 'PTV')
extracted_files = os.listdir(ptv_subdir)
print("Extracted files in PTV folder:", extracted_files)

# Step 3: Find the shapefile (.shp) in the 'PTV' folder
shapefile_name = [file for file in extracted_files if file.endswith('.shp')]
if shapefile_name:
    shapefile_path = os.path.join(ptv_subdir, shapefile_name[0])
    print(f"Shapefile found: {shapefile_path}")

    # Step 4: Load the train station shapefile
    try:
        train_station_gdf = gpd.read_file(shapefile_path)
        print("Train station shapefile loaded successfully.")
        print(train_station_gdf.head())
    except Exception as e:
        print(f"Error loading train station shapefile: {e}")
else:
    print("No shapefile found in the 'PTV' folder.")

Geocode to convert addresses to latitude and longtitude

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# File path for the rental data CSV
rental_data_path = '/home/Daniel Bi/project two/data/landing/rental_scrape.csv'

# Step 1: Load the rental data CSV
rental_df = pd.read_csv(rental_data_path)

# Step 2: Initialize Nominatim geocoder with retry logic
geolocator = Nominatim(user_agent="rental_geocoder", timeout=10)

# Create a session with retry settings to handle temporary errors
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Step 3: Function to geocode addresses with retries
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return (None, None)

# Step 4: Apply geocoding function to the 'Address' column with retry logic
rental_df['coordinates'] = rental_df['Address'].apply(lambda address: geocode_address(address))

# Step 5: Split coordinates into 'latitude' and 'longitude' columns
rental_df['latitude'] = rental_df['coordinates'].apply(lambda x: x[0])
rental_df['longitude'] = rental_df['coordinates'].apply(lambda x: x[1])

# Step 6: Save the DataFrame with geocoded coordinates to a new CSV
rental_df.to_csv('/home/Daniel Bi/project two/data/landing/rental_with_coordinates.csv', index=False)

# Optional: Display the first few rows to verify
print(rental_df[['Address', 'latitude', 'longitude']].head())

# Add a small delay between requests to avoid being blocked by Nominatim API
time.sleep(1)

Calculate distance to cloest train station